In [2]:
import grid2op
import torch
import warnings
from torch.utils.data import DataLoader
from dreamer.modules.encoder import Encoder
from dreamer.modules.decoder import Decoder
from dreamer.modules.recurrentModel import RecurrentModel
from dreamer.modules.dynamics import DynamicPredictor
from collections import defaultdict
from dreamer.modules.rewardModel import RewardPredictor
from dreamer.modules.continueModel import ContinuousPredictor
from dreamer.Utils.utils import Config
from dreamer.Utils.converter import ActionConverter
from dreamer.modules.rssm import RSSM
from dreamer.modules.worldModel import WorldModel, Trainer
from dreamer.Utils.dataset import GrdiDataset, load_npz_files_from_folder
warnings.filterwarnings('ignore')

c:\Users\Ernest\.conda\envs\l2rpn-test\lib\site-packages\grid2op\Backend\pandaPowerBackend.py:36: UserWarning: Numba cannot be loaded. You will gain possibly massive speed if installing it by 
	c:\Users\Ernest\.conda\envs\l2rpn-test\python.exe -m pip install numba

  warnings.warn(


In [3]:
d = torch.device("cuda" if torch.cuda.is_available() else "cpu")
d

device(type='cuda')

In [4]:
env = grid2op.make("l2rpn_case14_sandbox")
obs = env.reset()

In [5]:
config = Config.from_yaml('config.yml')

# Encoder test

In [13]:
obs = torch.tensor(obs.to_vect(), device=d)
obs = obs.unsqueeze(0)

In [14]:
e = Encoder(config)
e.to(device=d)


hidden_dim = config.hidden_dim
obs_0 = torch.rand(16, config.input_dim, device=d)
h_0 = torch.zeros(16, hidden_dim, device=d)


input_to_encoder = torch.cat((obs_0, h_0), dim=-1)
print(f"input shape : {input_to_encoder.shape}")
z, dist = e(input_to_encoder)
z.shape

input shape : torch.Size([16, 595])


torch.Size([16, 64])

# Decoder Test

In [ ]:
decoder = Decoder(config=config)

re_obs = decoder(z, h_0)
re_obs.size()

# Recurrent Model Test

In [ ]:
t_obs = torch.rand(32, config.input_dim, device=d)
z_0 = torch.randn(32, config.latent_dim, device=d)           # Initial latent state from encoder (randomly initialized here)
a_0 = torch.zeros(32, config.action_dim, device=d)           # Initial action is set to zero
h_0 = torch.zeros(32, config.hidden_dim, device=d)

In [ ]:
rec = RecurrentModel(config=config)
rec.to(device=d)
h_1 = rec(z_0, a_0, h_0)
print("Shape of h_1:", h_1.shape)  # Should be (hidden_dim,)

Shape of h_1: torch.Size([1, 128])


# Dynamic Model

In [ ]:
d_model = DynamicPredictor(config=config)
dist, next_obs = d_model(h_1)
next_obs.shape

torch.Size([1, 64])

# Reward Predictor

In [ ]:
e = Encoder(config)
e.to(device=d)
re = RewardPredictor(config)
re.to(device=d)

input_to_encoder = torch.cat([t_obs, h_0], dim=-1)

z_hat, posterior = e(input_to_encoder)
r_0 = re(z_hat, h_0)

# Continue Predictor

In [ ]:
c = ContinuousPredictor(config).to(device=d)
flag = c(z_hat, h_0)

In [ ]:
flag.shape

torch.Size([32, 1])

# RSSM Test

In [ ]:
rssm = RSSM(config=config)
rssm.to(device=d)
z_t, z, h, dist, dynamic_dist = rssm(t_obs, h_0, a_0)      #obs, hidden_state, action